In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
from urllib.request import urlopen
from selenium import webdriver

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Nasa Mars News

In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
news_title = soup.find("div", class_="content_title").a.text
news_p = soup.find("div", class_="rollover_description_inner").text
    
print('-------------')
print(news_title)
print(news_p)

-------------

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover


NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 



# JPL Mars Space Images - Featured Image

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
url ='https://www.jpl.nasa.gov/spaceimages'
browser.visit(url)

In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [8]:
relative_path = soup.find('div', class_='carousel_items')('article')[0]['style']
relative_path

"background-image: url('/spaceimages/images/wallpaper/PIA17171-1920x1200.jpg');"

In [9]:
get_image_url = relative_path.split("'")
get_image_url

['background-image: url(',
 '/spaceimages/images/wallpaper/PIA17171-1920x1200.jpg',
 ');']

In [10]:
unclean_image_url = get_image_url[1]
unclean_image_url

'/spaceimages/images/wallpaper/PIA17171-1920x1200.jpg'

In [11]:
symbol_removed = unclean_image_url.replace(');', '')
symbol_removed

'/spaceimages/images/wallpaper/PIA17171-1920x1200.jpg'

In [12]:
main_page = 'https://www.jpl.nasa.gov'

In [13]:
featured_image_url = main_page + symbol_removed
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17171-1920x1200.jpg


# Mars Weather

In [14]:
url = 'https://twitter.com/marswxreport'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [15]:
weather = soup.find_all('div', class_='js-tweet-text-container')

for tweets in weather:
    mars_weather = tweets.find('p').text
    
    if 'sol' and 'pressure' in mars_weather:
        print(f"{mars_weather}")
        break
    else:
        pass

InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)
winds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)
pressure at 6.30 hPapic.twitter.com/2h0LKoSQrJ


In [16]:
new_mars_weather = mars_weather.split('pic')
new_mars_weather

['InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)\nwinds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)\npressure at 6.30 hPa',
 '.twitter.com/2h0LKoSQrJ']

In [17]:
cleaned_tweet = new_mars_weather[0].replace('\n', '')
cleaned_tweet

'InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)winds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)pressure at 6.30 hPa'

# Mars Facts

In [18]:
url = 'https://space-facts.com/mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
mars_facts = soup.find('table')
mars_facts

<table class="tablepress tablepress-id-p-mars" id="tablepress-p-mars"><tbody><tr class="row-1 odd"><td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × 10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr class="row-5 odd"><td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/></td></tr><tr class="row-7 odd"><td cla

In [20]:
table_rows = mars_facts.find_all('tr')

df_data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    df_data.append(row)
    scrape_table = pd.DataFrame(df_data, columns=['Description', 'Value'])
    
scrape_table

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [21]:
scrape_table.reset_index()
new_table = scrape_table.set_index('Description')
new_table

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [22]:
new_table.to_html('Mars_facts_tb.html', index=True)

# Mars Hemispheres

In [23]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [24]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [25]:
hemisphere_image_urls = []
main_url = 'https://astrogeology.usgs.gov'

links = soup.find_all('div', class_='item')

In [26]:
for link in links:
    img_url = link.find('a', class_='itemLink product-item')['href']
    title = link.find('h3').text
    final_url = main_url + img_url
    browser.visit(final_url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img_wd = soup.find('div', class_='downloads')
    img_link = img_wd.find('a')['href']
    hemisphere_image_urls.append({'title':title, 'image_url':img_link})
    

In [27]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]